# Extracting Tweets and Comments

In [ ]:
import tweepy
import json
import urllib.request, urllib.error, urllib.parse
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
ckey = 'W8m2pHHclp2VGsewYmggATIXF'
csecret = '9gNOSYkQIayUIkb0bSwkAznSq4lizRDkcMhLkqRYWhjP0IV9oi'
atoken = '1062248841443971072-PyxXIA0Cxjyh6j4XtbVsNiosS6sXsx'
asecret = 'ipwgD5lVfmpJTv83OL11tepOLPvlI3KVjFG57quhJbjTc'

# the authentication process for tweepy
auth = tweepy.OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
api = tweepy.API(auth)
#print(api)

In [ ]:
dow_jones = pd.read_csv('dow_jones.csv')
ticker = dow_jones['Ticker'].tolist()
companies = []
for let in ticker:
    let = '$'+let
    companies.append(let)         #adding $ to ticker symbols

In [ ]:
tweet_text = []
tweet_id = []
tweet_created = []
company = []
company_comment = []
tweet_comment_date = []
tweet_comment = []

In [ ]:
counter = 0
for let in companies:
    counter +=1
    maxid = 99999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999
    for k in range(0 , 10):

        if k == 0:
            #get the initial tweets:
            tweets = api.search( q = let , count = 100)
        else:
            tweets = api.search(q = let , count = 100, max_id = maxid)

        maxid = 99999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999

        for i in range(len(tweets)):
            if tweets[i].id < maxid:
                maxid = tweets[i].id

                a=tweets[i]._json['user']['screen_name']   #@username
                b=tweets[i].id                             #tweet id

                link = 'https://twitter.com/'+a+'/status/'+str(b)
                html = requests.get(link)
                soup = BeautifulSoup(html.text, 'lxml')               
                #extract the text from these tweets
                tweet_text.append(tweets[i].text)                 #extracting tweets
                tweet_id.append(tweets[i].id)
                tweet_created.append(str(tweets[i].created_at))   #extracting tweet date
                company.append(let)
                print(len(company),counter,let)
                t=soup.find_all('div', class_='content')[1:]
                for i in t:
                    date = i.find('a', class_='tweet-timestamp js-permalink js-nav js-tooltip')
                    comment = i.find('p', class_='TweetTextSize js-tweet-text tweet-text')
                    if comment is not None and date is not None:
                        date = date.getText()                   #extractng reply date
                        comment = comment.getText()             #extracting tweet reply
                        tweet_comment.append(comment)          
                        tweet_comment_date.append(date)
                        company_comment.append(let)

        maxid = maxid - 1
            

In [ ]:
tweets = pd.DataFrame({'Company':company,'Date':tweet_created,'Tweet':tweet_text})

In [ ]:
comments = pd.DataFrame({'Company':company_comment,'Date':tweet_comment_date, 'Tweet':tweet_comment})

In [ ]:
tweets.to_csv('stocktweets.csv', index=False)
comments.to_csv('stockComments.csv',index=False)